### `tes-thermo`
---

In [1]:
from tes_thermo.thermo_agent import ThermoAgent
from tes_thermo.utils import VectorSearch
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings
from dotenv import load_dotenv
import os

In [2]:
load_dotenv(override=True)

# example using Azure OpenAI services
# Make sure to set the environment variables AZURE_OPENAI_API_KEY, AZURE_OPENAI_ENDPOINT,
# LLM_DEPLOYMENT_MODEL, EMBEDDING_DEPLOYMENT_MODEL, LLM_API_VERSION, and EMBEDDING_API_VERSION
# in your .env file or your environment before running this code.

class AzureServices():
    def __init__(self):
        self.AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
        self.AZURE_OPENAI_ENDPOINT= os.getenv("AZURE_OPENAI_ENDPOINT")
        self.DEPLOYMENT_MODEL = "gpt-4.1-mini"
        self.EMBEGGIND_MODEL = os.getenv("EMBEDDING_DEPLOYMENT_MODEL")
        self.LLM_VERSION = os.getenv("LLM_API_VERSION")
        self.EMBEDDING_VERSION = os.getenv("EMBEDDING_API_VERSION")

    def get_default_llm(self):
        return AzureChatOpenAI(openai_api_version=self.LLM_VERSION,
                                azure_endpoint=self.AZURE_OPENAI_ENDPOINT,
                                openai_api_key=self.AZURE_OPENAI_API_KEY,
                                azure_deployment=self.DEPLOYMENT_MODEL,
                                temperature=0)
    
    def get_default_embedding(self):
        return AzureOpenAIEmbeddings(api_key=self.AZURE_OPENAI_API_KEY,
                                        api_version=self.EMBEDDING_VERSION,
                                        deployment=self.EMBEGGIND_MODEL,
                                        azure_endpoint=self.AZURE_OPENAI_ENDPOINT)

services = AzureServices()
llm = services.get_default_llm()
embedding_model = services.get_default_embedding()

In [3]:
%cd ..

/Users/mitoura/Desktop/Repositories/tes-thermo


In [4]:
vsearch = VectorSearch.from_documents(
    document_paths=['article.pdf'],  # replace with your document paths (pdf only...)
    embedding=embedding_model
)

Index created successfully with 35 chunks.


In [5]:
agent = ThermoAgent(llm = llm,
                    vsearch = None,
                    embedding_model= embedding_model)

In [7]:
prompt = """
Simule o processo de hidrogenacao de acido laurico considernado os seguintes compoenentes:
'water','carbon monoxide', 'carbon dioxide', 'hydrogen', 'C12H24O2','143-07-7', 'C11H24', 'C12H26', 'ethane', 'propane', 'butane', 'CH4'


Simule o processo de 400 a 600 K, 10 a 50 bar considerando a alimentacao composta de 3 mols de H2 e 1 mol de 143-07-7.

O objetivo e avaliar as rotas adotadas, como referencia, consulte o artigo indicado.
Mostre os resultados como tabela.
"""
res = agent.chat(prompt=prompt)
print(res['output'])

Aqui está a tabela resumida dos resultados da simulação do processo de hidrogenação do ácido láurico (143-07-7) com alimentação de 3 mols de H2 e 1 mol de 143-07-7, variando temperatura de 400 a 600 K e pressão de 10 a 50 bar. As concentrações são em mols:

| Temperatura (K) | Pressão (bar) | water       | CO          | CO2         | H2          | 143-07-7    | C11H24      | C12H26      | ethane      | propane     | butane      | CH4         |
|-----------------|---------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|-------------|
| 400             | 10            | 7.5e-9      | 7.5e-3      | 0.996       | 4.4e-8      | 6.6e-9      | 0.38        | 0.21        | 0.15        | 0.18        | 0.13        | 2.95        |
| 400             | 27.8          | 2.9e-8      | 1.8e-3      | 0.999       | 4.4e-8      | 2.3e-8      | 0.38        | 0.21        | 0.15        | 0.19        | 0.14      